In [1]:
from Prompts import prompter
from dotenv import load_dotenv
import os
from google import genai

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

client = genai.Client()

In [2]:
import json

# Path to your JSON file
file_path = "Datasets/network_analysis_1/na1.json"

# Load the JSON content
with open(file_path, "r") as f:
    data = json.load(f)

# Extract parts from the JSON
scenario = data["scenario"]
tools = data["tools"]
questions = data["questions"]
file=data["files"]



In [8]:
Answer_list=[]
for i in range(0, len(questions)):

    j = 0
    max_attempts = 4 #max per question
    history=[]  #here only used to store (thought, action, observation)
    
    prompt = prompter.prompt_manual_thought(i, scenario, tools, file, questions, history) #prompt that trigger thought
    
    response = client.models.generate_content(    # generates thought
        model="gemini-2.5-pro", contents=prompt
    )
    thought=response.text

    print(f"🧠 Thought generated for question {i+1}: {thought}")
    prompt=prompter.prompt_manual_action(i,scenario,tools, file,questions, history, thought) #prompt to generate action

    response = client.models.generate_content(    # generates command based on the thought
        model="gemini-2.5-pro", contents=prompt
    )
    action=response.text
    
    print(f"🛠️ Please execute this command for question {i+1}: {action}")
    
    while j < max_attempts:

        user_command_output = input("""Please paste the command's output here or type "0" to skip question :\n""")
        if user_command_output== "0":
            print(f"⛔ Skipped question {i+1}.")
            Answer_list.append("Skipped")
            break
        #user_command_output = user_command_output[:1000]   # Limit to 500 characters(long outputs ruins agent's performance+ take long time)
        history.append({"previous Thought": [thought],"previous command": [action], "previous command output": [user_command_output]})
        print(f"📜 {history}")
        prompt = prompter.prompt_manual_thought(i, scenario, tools, file, questions, history)  # prompt to generate next thought
        
        response = client.models.generate_content(       # generates next thought based on the command output
            model="gemini-2.5-pro",
            contents=prompt
        )
        thought = response.text

        print(f"🧠 Thought generated for question {i+1}: {thought}")

        prompt=prompter.prompt_manual_action(i,scenario,tools, file,questions, history, thought) #prompt to generate action

        response = client.models.generate_content(    # generates command based on the thought
            model="gemini-2.5-pro", contents=prompt
        )
        action=response.text
        
        

        if "final answer:" in action.lower():
            text = action.strip()
            answer_part = text.lower().split("final answer:")[-1].strip()
            print(f"✅ Answer to the question {i+1}: {answer_part}")
            Answer_list.append(answer_part)
            break
        else:
            print(f"🛠️ Please execute this command for question {i+1}: {action}")            
        
        j += 1
    if j == max_attempts:
        Answer_list.append("Answer not found")  

print("✅ Task finished.")        


🧠 Thought generated for question 1: To identify the private IP address of the infected host, I need to list the IP addresses involved in the network traffic captured in the PCAP file. A good first step is to get a summary of all IP conversations. This will show which IP addresses are communicating with each other. By inspecting this list, I can look for an IP address that falls into the private (RFC 1918) address space (10.0.0.0/8, 172.16.0.0/12, 192.168.0.0/16). This private IP is almost certainly the infected internal host. The `tshark` tool has a feature to display IP conversations, which is perfect for this purpose.
🛠️ Please execute this command for question 1: command: tshark -r Datasets/network_analysis_1/traffic-with-dridex-infection.pcap -q -z conv,ip
📜 [{'previous Thought': ['To identify the private IP address of the infected host, I need to list the IP addresses involved in the network traffic captured in the PCAP file. A good first step is to get a summary of all IP convers

In [9]:
print(Answer_list)


['10.11.27.101', 'oiioiashdqbwe.rar', 'cochrimato.com', 'http://95.181.198.231/oiioiashdqbwe.rar', '185.244.150.230']


In [10]:
#saves all the answers to a file for evaluation
base_name = os.path.splitext(os.path.basename(file_path))[0]

output_file = os.path.join("Evaluation","manual","ReAct", f"{base_name}.txt")

with open(output_file, 'w') as f:
    for answer in Answer_list:
        f.write(answer + "\n")

print(f"Answers saved to {output_file}")

Answers saved to Evaluation/manual/ReAct/na1.txt
